<ul>
<li>ovarian cancer</li>
<li>locally available bio-markers</li>
<li>train on dataset_main, benchmark on OOB_C</li>
<li>age between 40-70</li>
<li>marker normalization based on "Normal"s median</li>
<li>distribution based classification</li>
</ul>

# Modules and Data loading

In [2]:
import sys
import json

with open('../paths', 'r') as f:
    globals().update(json.loads(f.read()))

sys.path.append(path_Modules_Product_)
sys.path.append(path_Modules_Toolbox_)

In [3]:
import pandas as pd
import numpy as np
from scipy import stats as spstat
from sklearn import covariance as skcov
from sklearn import preprocessing as skprep
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [4]:
import data_processing as myprep
import handles as myhandles
import Plots as IcePlot

In [5]:
dataset = pd.read_csv(path_data_proprocessed_ + 'dataset/ds_main.csv',
                      index_col=0)
dataset = dataset.loc[(50 <= dataset.s_Age) & (dataset.s_Age <= 70)]
markers_ = [
    'IL6', 'IL8', 'PROLACTIN', 'NSE', 'CA15-3', 'CA19-9', 'CEA', 'AFP', 'HE4',
    'CA125'
]
markers = [f'p_{x}' for x in markers_]
markers_l = [f'l_{x}-limit' for x in markers_]

In [6]:
p_rows_2 = 5

# Basic Analysis

In [7]:
ds_ = dataset.loc[dataset.s_Sex=='F'].copy()
ds_ = dataset.copy()
ds_[markers] = np.log(ds_[markers])
ds_norm = ds_.loc[ds_.s_Tumor == 'Normal']
ds_affe = ds_.loc[ds_.s_Tumor == 'Ovary']
_, subs = plt.subplots(p_rows_2, 2, figsize=(15, 10))
subs = subs.flatten()
for c, (m_l, m) in enumerate(zip(markers_l, markers)):
    sub = subs[c]
    plt.sca(sub)
    IcePlot.linespace_kde(ds_norm,
                          m,
                          title=m,
                          ax=sub,
                          grid=True,
                          label='Normal')
    IcePlot.linespace_kde(ds_affe,
                          m,
                          title=m,
                          ax=sub,
                          grid=True,
                          label='Ovarian')
    plt.legend()
plt.suptitle('whole dataset', y=1)
plt.tight_layout()

In [8]:
ds_ = dataset.loc[dataset.s_Tumor == 'Normal'].copy()
ds_[markers] = np.log(ds_[markers])
_, subs = plt.subplots(p_rows_2, 2, figsize=(15, 10))
subs = subs.flatten()
for c, (m_l, m) in enumerate(zip(markers_l, markers)):
    sub = subs[c]
    plt.sca(sub)
    #
    IcePlot.linespace_kde(ds_, m, title=m, ax=sub, grid=True)
    #
    v = ds_[m].median()
    plt.vlines(v, *plt.ylim(),color='r',label='median')
    #
    v = ds_.loc[ds_[m_l] == 1, m].values
    plt.vlines(v, *plt.ylim(),linestyles='dotted',label='limit')
    #
    limit_str = ds_[m_l].value_counts(normalize=1, sort=False).to_string()
    limit_str = 'out of limits:\n' + limit_str
    plt.text(1,
             1,
             limit_str,
             transform=sub.transAxes,
             verticalalignment='top',
             horizontalalignment='right')
    plt.legend(loc='lower right')
plt.suptitle('Normals & limits', y=1)
plt.tight_layout()

In [9]:
ds_ = dataset.loc[dataset.s_Tumor == 'Normal'].copy()
ds_[markers] = np.log(ds_[markers])
_, subs = plt.subplots(p_rows_2, 2, figsize=(15, 10))
subs = subs.flatten()
for c, (m_l, m) in enumerate(zip(markers_l, markers)):
    sub = subs[c]
    plt.sca(sub)
    ds__ = ds_.loc[ds_[m_l] == 0]
    IcePlot.linespace_kde(ds__, m, title=m, ax=sub, grid=True, n_inds=500)
    count_str = f'samples: {len(ds__)}'
    plt.text(1,
             1,
             count_str,
             transform=sub.transAxes,
             verticalalignment='top',
             horizontalalignment='right')
plt.suptitle('Normals (limits dropped)', y=1)
plt.tight_layout()

# Median based normalization

In [10]:
ds_ = dataset.loc[(dataset.s_Sex == 'F')
                  & (dataset.s_Tumor == 'Normal')].copy()
ds_[markers] = np.log(ds_[markers])
_, subs = plt.subplots(p_rows_2, 2, figsize=(15, 18))
subs = subs.flatten()
for c, (m, m_l) in enumerate(zip(markers, markers_l)):
    sub = subs[c]
    plt.sca(sub)
    #
    ds__ = ds_.loc[ds_[m_l] == 0]
    plt.scatter(ds__.s_Age, ds__[m], color='b', label='normal samples')
    ds__ = ds_.loc[ds_[m_l] == 1]
    plt.scatter(ds__.s_Age, ds__[m], color='r', label='ooll samples')
    ds__ = ds_.loc[ds_[m_l] == 2]
    plt.scatter(ds__.s_Age, ds__[m], color='y', label='ooul samples')
    #
    tmp = ds_[['s_Age', m]].groupby('s_Age').median()
    plt.plot(tmp.index, tmp[m], color='b', label='Age median')
    #
    q = 10
    tmp = ds_[['s_Age', m]].copy()
    tmp['Age_grp_med'] = pd.qcut(tmp.s_Age, q)
    tmp_2 = tmp[['Age_grp_med', m]].groupby('Age_grp_med').median()
    tmp_2['Age_mean'] = pd.Series(tmp_2.index).apply(
        lambda x: (x.left + x.right) / 2).values.astype(float)
    plt.plot(tmp_2.Age_mean.values,
             tmp_2[m].values,
             color='c',
             label=f'Age_Q median:{q}')
    
    tmp=np.polyfit(tmp_2.Age_mean,tmp_2[m],1)
    tmp = np.poly1d(tmp)
    x=np.linspace(ds_.s_Age.min(),ds_.s_Age.max(),10)
    y = tmp(x)
    plt.plot(x,y,label='fitted',color='r')
    #
    plt.grid()
    plt.ylabel(m)
    plt.legend()
    ax = plt.twinx(sub)
    tmp = ds_.s_Age.value_counts()
    plt.bar(tmp.index, tmp.values, color='g', alpha=0.3)
plt.tight_layout()

No tough trends in important features. leaving it at least by now

In [11]:
ds_ = dataset.loc[dataset.s_Sex == 'F'].copy()
ds_[markers] = np.log(ds_[markers])
ds_norm = ds_.loc[ds_.s_Tumor == 'Normal'].copy()
ds_affe = ds_.loc[ds_.s_Tumor == 'Ovary'].copy()
meds = ds_norm[markers].median().values
ds_norm[markers] -= meds
ds_affe[markers] -= meds
_, subs = plt.subplots(p_rows_2, 2, figsize=(15, 18))
subs = subs.flatten()
for c, m in enumerate(markers):
    sub = subs[c]
    plt.sca(sub)
    IcePlot.linespace_kde(ds_norm, m, title=m, label='Normal')
    IcePlot.linespace_kde(ds_affe, m, label='Ovarian')
    plt.grid()

# Naive distribution comparison

In [23]:
fit_markers = ['p_HE4','p_CA125','p_OPN']
ds_ = dataset.loc[dataset.s_Sex == 'F'].copy()
ds_[fit_markers] = np.log(ds_[fit_markers])
trans = skprep.StandardScaler().fit(
    ds_.loc[ds_.s_Tumor == 'Normal', fit_markers])
ds_[fit_markers] = trans.transform(ds_[fit_markers])
ds_norm = ds_.loc[ds_.s_Tumor == 'Normal'].copy()
ds_affe = ds_.loc[ds_.s_Tumor == 'Ovary'].copy()

loc_norm = np.zeros(len(fit_markers))
loc_affe = ds_affe[fit_markers].median()
cov_norm = skcov.EmpiricalCovariance(assume_centered=True).fit(
    ds_norm[fit_markers]).covariance_
cov_affe = skcov.EmpiricalCovariance(assume_centered=False).fit(
    ds_affe[fit_markers]).covariance_
dist_norm = spstat.multivariate_normal(loc_norm, cov_norm)
dist_affe = spstat.multivariate_normal(loc_affe, cov_affe)
ds__ = pd.concat((ds_norm, ds_affe), axis=0)
ds__['target'] = 0
ds__.loc[ds__.s_Tumor == 'Ovary', 'target'] = 1
x = ds__[fit_markers].values
ds__['lr'] = dist_affe.pdf(x) / dist_norm.pdf(x)
ds__['llr'] = np.log(ds__['lr'])
plt.figure(figsize=(15, 7))
IcePlot.binroc(ds__.target.values, ds__.lr.values)

## OOB

In [24]:
dataset_oob = pd.read_csv(path_data_proprocessed_ + 'dataset/ds_OOB_C.csv',index_col=0)
dataset_oob['p_HE4']= dataset_oob['p_HE4_L'].values
dataset_oob.s_Time2dx = dataset_oob.s_Time2dx.replace(np.nan,-1)
dataset_oob = dataset_oob.dropna(subset=fit_markers)

In [25]:
ds_ = dataset_oob.copy()
ds_ = ds_.loc[ds_.s_Time2dx<180]
ds_[fit_markers] = np.log(ds_[fit_markers])
trans = skprep.StandardScaler().fit(ds_.loc[ds_.s_Tumor=='Normal',fit_markers])
ds_[fit_markers] = trans.transform(ds_[fit_markers])
ds_['target'] = 0
ds_.loc[ds_.s_Tumor == 'Ovary', 'target'] = 1
x = ds_[fit_markers].values
ds_['lr'] = dist_affe.pdf(x) / dist_norm.pdf(x)
plt.figure(figsize=(15,7))
IcePlot.binroc(ds_.target.values, ds_.lr.values)

# bias inducing features eliminated

In [1]:
# todo : bias inducing features drop using dist. comparison